In [ ]:
import xml.etree.ElementTree as ET
import os
import pathlib

from unidecode import unidecode

In [ ]:
def get_dados_basicos(xml):
    tree = ET.parse(xml)

    root = tree.getroot()

    string = ""

    for child in root.findall('DADOS-GERAIS'):
        string+="Nome: "+child.attrib["NOME-COMPLETO"]+"\n\n"

    return string;

        
def get_artigos(xml):
    tree = ET.parse(xml)

    root = tree.getroot()

    conteudo = ""

    for child in root.findall("PRODUCAO-BIBLIOGRAFICA"):
        for artigos in child.findall("ARTIGOS-PUBLICADOS"):
            for artigo in artigos.findall("ARTIGO-PUBLICADO"):
                string = ""
                for dados_basicos in artigo.findall("DADOS-BASICOS-DO-ARTIGO"):

                    string += "\t<Artigo>\n"

                    attr = dados_basicos.attrib
                    doi = ""

                    for i in attr.keys():
                        field_name = convert_attrib_name_to_string(i)
                        field = convert_attrib_name_to_string(attr[i])
                        if "Home page do trabalho" in field_name:
                            field = field.replace("[", "").replace("]", "")

                            if "doi:" in field:
                                aux = field.split("doi:")
                                field = aux[0]
                                doi = aux[1]
                        
                        if "Doi" in field_name and not field and doi:
                            field = doi
                            
                        string += "\t\t"+field_name+": "+field+"\n" if field else ""


                string += "\n"


                for detalhamento in artigo.findall("DETALHAMENTO-DO-ARTIGO"):
                    
                    attr = detalhamento.attrib
                    
                    for i in attr.keys():
                        field_name = convert_attrib_name_to_string(i)
                        field = convert_attrib_name_to_string(attr[i])
                        string += "\t\t"+field_name+": "+field+"\n" if field else ""


                string += ("\n")


                string += "\t\tAutor(es) do artigo: "

                for autores in artigo.findall("AUTORES"):
                    string += autores.attrib["NOME-COMPLETO-DO-AUTOR"]+", "
                
                string = string.rstrip(", ")

                string += "\n\n"

                for palavras_chave in artigo.findall("PALAVRAS-CHAVE"):
                    keywords = palavras_chave.attrib
                    for i in keywords.keys():
                        field_name = convert_attrib_name_to_string(i)
                        field = convert_attrib_name_to_string(keywords[i])
                        string += "\t\t"+field_name+": "+field+"\n" if field else ""
                
                    string += "\n"            

                for areas_conhecimento in artigo.findall("AREAS-DO-CONHECIMENTO"):
                    areas = [elem.tag for elem in areas_conhecimento.iter() if elem is not areas_conhecimento]
                    for i in range(len(areas)):
                        for area in areas_conhecimento.findall(areas[i]):
                            for i in area.attrib.keys():
                                field_name = convert_attrib_name_to_string(i)
                                field = convert_attrib_name_to_string(area.attrib[i])
                                string += "\t\t"+field_name+": "+field+"\n" if field else ""
                    
                    string += "\n"

                
                conteudo += string
                

    return conteudo

def gerar_arquivo(saida, xml):
    content = get_dados_basicos(xml)
    content += get_artigos(xml)

    saida.write(content)

def convert_attrib_name_to_string(attrib_name):
    return attrib_name.lower().capitalize().replace("-", " ")

def xml_get_first_name(xml):
    tree = ET.parse(xml)
    root = tree.getroot()
    
    nome = ""

    for child in root.findall('DADOS-GERAIS'):
        nome = child.attrib["NOME-COMPLETO"]
    
    primeiro_nome = nome.split(" ")
    
    return primeiro_nome[0]

In [ ]:
diretorio_entrada = "./xml/"
diretorio_saida = "./saidas/"

if not os.path.exists(diretorio_entrada):
    os.makedirs(diretorio_entrada)


if not os.path.exists(diretorio_saida):
    os.makedirs(diretorio_saida)


saida_geral = open(diretorio_saida+"todos_curriculos.txt", 'w')


for filename in os.listdir(diretorio_entrada):
    
    f = os.path.join(diretorio_entrada, filename)

    if os.path.isfile(f):
        
        if(pathlib.Path(f).suffix == '.xml'):
            
            xml = ""+f

            nome_do_arquivo = diretorio_saida+"curriculo_"+unidecode(xml_get_first_name(xml).lower())+".txt"
            
            print("Gerando arquivo: "+nome_do_arquivo)

            saida_individual = open(nome_do_arquivo, "w")

            gerar_arquivo(saida_individual, xml)
            gerar_arquivo(saida_geral, xml)

            saida_individual.close()

saida_geral.close()


In [ ]:
'''
tree = ET.parse("./xml/curriculo_2.xml")

root = tree.getroot()
arquivo = open("./teste.txt", 'w')
elemList = []

for child in root.iter():
    for i in child.attrib.keys():
        field_name = convert_attrib_name_to_string(i)
        field = convert_attrib_name_to_string(child.attrib[i])
        arquivo.write(field_name+": "+field+"\n" if field else "")
    arquivo.write("\n")


arquivo.close()
'''